# Preprocessors  
This notebook makes use of the findings of the explorations notebooks, and using the `tools/save4later.py` module, creates and stores the preprod's appropriately for the models to load seamlessly.  

First, let's import the `tools/`

In [1]:
# General imports
import numpy as np
import pandas as pd

# Import our tools
import sys
sys.path.append('/Users/will/Github/facial-keypoint-detection/scripts')

# Import submit module from our tools subfolder
from tools import getdata, save4later

And let's import all our data

In [2]:
# Load data (no dev)
_loaded = getdata.load_data(0, test=False, nonas=False)

FEATURES = _loaded['features']
print 'Number of features:', len(FEATURES)

train_data = _loaded['training']['data']
train_labels = _loaded['training']['labels']
print 'Training dataset size: ', train_data.shape

Number of features: 30
Training dataset size:  (7049,)


## Contents  

* **Noise Cancelling**:
   * Mask Faces  


* **Edge Detection**:
   * Sobel filter  
   * Laplace transform  


* **Noise Reduction**:
   * Gaussian blur  
   * Mean-shift  
 

* **Finding Contours**:  
   * Histogram of Oriented Gradients  
   * Watershed algorithm  

## 1. Mask Faces
Using `explorations/face-detection.ipynb` creates the masked-face preprod.  

Process:  
* All the images are run through OpenCV's face-detector (*see the exploration notebook for more detail and references on this algorithm*)  
* Images for which just one face is detected are masked (set all pixels outside the detected face to 0)  

*IMPORTANT NOTE* - the base `environment.yml` does not include installation of OpenCV2, since it is an optional branch in this project. In order to get the build including OpenCV refer to with `/adv-envs/environment-ocv.yml`.  


In [3]:
import cv2

In [4]:
CASC_PATH = '../haarcascade_frontalface_default.xml'

# Create the haar cascade classifier
faceCascade = cv2.CascadeClassifier(CASC_PATH)

def detect_face(img, min_face_size):
    ''' Detect faces in the image'''
    face = faceCascade.detectMultiScale(
        img,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(min_face_size, min_face_size),
        flags = cv2.cv.CV_HAAR_SCALE_IMAGE)
    return face

In [5]:
def mask_faces(faces):
    ''' Masked faces detected '''
    masked = []
    
    for i,_face in enumerate(faces):
        old_img = train_data[_face].reshape(96,96)

        # detect face
        _face = detect_face(old_img, min_face_size=10)
                        
        # only crop if we detect one face
        if len(_face) == 1:
            x, y, w, h = _face[0]
            new_img = np.zeros_like(old_img)
            new_img[y:y+h, x:x+w] = old_img[y:y+h, x:x+w]
            
            masked.append(new_img.flatten())
            
        else:
            masked.append(old_img.flatten())
        
    return masked

Let's **mask** all the images now!

In [6]:
masked_train_data = mask_faces( range(len(train_data)) )

And now save using the `save4later` tool ...

In [7]:
save4later.save_preprod(masked_train_data, 'masked',
                        description='Masked data (removed background of all faces)')

In [8]:
save4later.list_preprods()


    PREPROC'D DATASETS
 Total: 2

  + masked.pk : Masked data (removed background of all faces)
  + test.pk : This is a test


Great, there it is! On to the next one then...